In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import names
from collections import defaultdict

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/apple/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/apple/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/apple/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

In [3]:
# newsgroups.data

In [4]:
data = pd.Series(newsgroups.data)

In [5]:
df = pd.DataFrame(data)

In [6]:
df.columns = ['Text'] + df.columns.tolist()[1:]

In [7]:
df['Target'] = pd.Series(newsgroups.target)

In [8]:
df

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [9]:
df['Target'].nunique()

20

In [10]:
df['Target'].isnull().value_counts()

False    18846
Name: Target, dtype: int64

In [11]:
# email_tokenizer = RegexpTokenizer(r'\S+@\S+(?:\.\S+)+')
tokenizer = RegexpTokenizer(r'[a-zA-Z0-9]+')


In [12]:


email_tokenizer = RegexpTokenizer(r'\S*@\S*\s?')

In [13]:
type(newsgroups.data)

list

In [14]:
df 

,Text,Target
0,\n\nI am sure some bashers of Pens fans are pr...,10
1,My brother is in the market for a high-perform...,3
2,\n\n\n\n\tFinally you said what you dream abou...,17
3,\nThink!\n\nIt's the SCSI card doing the DMA t...,3
4,1) I have an old Jasmine drive which I cann...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye)\nD...,13
18842,\nNot in isolated ground recepticles (usually ...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,\nWouldn't this require a hyper-sphere. In 3-...,1


In [15]:
common_names = names.words()



In [16]:
df.Text = df.Text.apply(lambda x: ' '.join([item for item in x.split() if item not in common_names]))


In [17]:
df

,Text,Target
0,I am sure some bashers of Pens fans are pretty...,10
1,My brother is in the market for a high-perform...,3
2,Finally you said what you dream about. Mediter...,17
3,Think! It's the SCSI card doing the DMA transf...,3
4,1) I have an old drive which I cannot use with...,4
...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye) DN...,13
18842,Not in isolated ground recepticles (usually an...,12
18843,I just installed a DX2-66 CPU in a clone mothe...,3
18844,Wouldn't this require a hyper-sphere. In 3-spa...,1


In [18]:
df['test'] = df.apply(lambda row: email_tokenizer.tokenize(row['Text']),axis=1)

In [19]:
df

,Text,Target,test
0,I am sure some bashers of Pens fans are pretty...,10,[]
1,My brother is in the market for a high-perform...,3,[]
2,Finally you said what you dream about. Mediter...,17,[]
3,Think! It's the SCSI card doing the DMA transf...,3,[]
4,1) I have an old drive which I cannot use with...,4,[]
...,...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye) DN...,13,[nyeda@cnsvax.uwec.edu ]
18842,Not in isolated ground recepticles (usually an...,12,[]
18843,I just installed a DX2-66 CPU in a clone mothe...,3,[]
18844,Wouldn't this require a hyper-sphere. In 3-spa...,1,[]


In [20]:
df

,Text,Target,test
0,I am sure some bashers of Pens fans are pretty...,10,[]
1,My brother is in the market for a high-perform...,3,[]
2,Finally you said what you dream about. Mediter...,17,[]
3,Think! It's the SCSI card doing the DMA transf...,3,[]
4,1) I have an old drive which I cannot use with...,4,[]
...,...,...,...
18841,DN> From: nyeda@cnsvax.uwec.edu (David Nye) DN...,13,[nyeda@cnsvax.uwec.edu ]
18842,Not in isolated ground recepticles (usually an...,12,[]
18843,I just installed a DX2-66 CPU in a clone mothe...,3,[]
18844,Wouldn't this require a hyper-sphere. In 3-spa...,1,[]


In [21]:
email_list_of_lists = df.test.tolist()

email_list = [item for item in email_list_of_lists if item != []]

emails = []
for x,y in enumerate(email_list):
    emails.extend(email_list[x])

In [22]:
email_list_2 = []
for email in emails:
    email_list_2.append(email.strip().rstrip(','))

In [23]:
email_list_2

['pmetzger@shearson.com',
 'grady@world.std.com',
 '<feustel@netcom.com>',
 '<1993Apr19.141959.4057@bnr.ca>',
 '(karr@cs.cornell.edu)',
 '<dwex@mtgzfs3.att.com>',
 '<xfree86@physics.su.oz.au>',
 '@66MHz',
 '@66MHz',
 '@150MHz',
 'rvenkate@uiuc.edu',
 '@InProceedings{Keirouz:et:al:90',
 '@InProceedings{Yamaguchi:Kimura:90',
 '@InProceedings{Chung:et:al:88',
 '@Article{Kimura:et:al:86',
 '@Article{Kimura:et:al:87',
 '@InProceedings{Chung:Schussel:89',
 '@Article{Pabon:et:al:92',
 '@Article{Kondo:90',
 '@InProceedings{Zalik:et:al:92a',
 'rogue@cs.neu.edu',
 '<1993Apr25.201129.1239@Princeton.EDU>',
 'fuchs@tsar.princeton.edu',
 '70423.600@compuserve.com',
 'paul@hsh.com)',
 'sthong@eniac.seas.upenn.edu',
 'sthong@eniac.seas.upenn.edu',
 '@',
 '@',
 '@',
 '(kph2q@virginia.edu)',
 'hughm@inmos.co.uk.',
 'rye@crayamid.cray.com',
 'keithley@apple.com',
 'keithley@apple.com',
 'keithley@apple.com',
 'keithley@apple.com',
 'baden@sys6626.bison.mb.ca',
 'baden@inqmind.bison.mb.ca',
 'baden@sys662

In [24]:
emails[1].strip()

'grady@world.std.com'

In [25]:
emails

['pmetzger@shearson.com',
 'grady@world.std.com ',
 '<feustel@netcom.com>',
 '<1993Apr19.141959.4057@bnr.ca> ',
 '(karr@cs.cornell.edu) ',
 '<dwex@mtgzfs3.att.com> ',
 '<xfree86@physics.su.oz.au>',
 '@66MHz ',
 '@66MHz ',
 '@150MHz ',
 'rvenkate@uiuc.edu',
 '@InProceedings{Keirouz:et:al:90, ',
 '@InProceedings{Yamaguchi:Kimura:90, ',
 '@InProceedings{Chung:et:al:88, ',
 '@Article{Kimura:et:al:86, ',
 '@Article{Kimura:et:al:87, ',
 '@InProceedings{Chung:Schussel:89, ',
 '@Article{Pabon:et:al:92, ',
 '@Article{Kondo:90, ',
 '@InProceedings{Zalik:et:al:92a, ',
 'rogue@cs.neu.edu ',
 '<1993Apr25.201129.1239@Princeton.EDU>, ',
 'fuchs@tsar.princeton.edu ',
 '70423.600@compuserve.com ',
 'paul@hsh.com)',
 'sthong@eniac.seas.upenn.edu ',
 'sthong@eniac.seas.upenn.edu ',
 '@ ',
 '@ ',
 '@ ',
 '(kph2q@virginia.edu) ',
 'hughm@inmos.co.uk. ',
 'rye@crayamid.cray.com ',
 'keithley@apple.com ',
 'keithley@apple.com ',
 'keithley@apple.com ',
 'keithley@apple.com ',
 'baden@sys6626.bison.mb.ca ',
 

In [26]:
df.Text = df.Text.apply(lambda x: ' '.join([item for item in x.split() if item not in email_list_2]))



In [27]:
df

,Text,Target,test
0,I am sure some bashers of Pens fans are pretty...,10,[]
1,My brother is in the market for a high-perform...,3,[]
2,Finally you said what you dream about. Mediter...,17,[]
3,Think! It's the SCSI card doing the DMA transf...,3,[]
4,1) I have an old drive which I cannot use with...,4,[]
...,...,...,...
18841,DN> From: (David Nye) DN> A neurology DN> cons...,13,[nyeda@cnsvax.uwec.edu ]
18842,Not in isolated ground recepticles (usually an...,12,[]
18843,I just installed a DX2-66 CPU in a clone mothe...,3,[]
18844,Wouldn't this require a hyper-sphere. In 3-spa...,1,[]


In [28]:
df['Text'] = df['Text'].str.lower()

In [29]:
df

,Text,Target,test
0,i am sure some bashers of pens fans are pretty...,10,[]
1,my brother is in the market for a high-perform...,3,[]
2,finally you said what you dream about. mediter...,17,[]
3,think! it's the scsi card doing the dma transf...,3,[]
4,1) i have an old drive which i cannot use with...,4,[]
...,...,...,...
18841,dn> from: (david nye) dn> a neurology dn> cons...,13,[nyeda@cnsvax.uwec.edu ]
18842,not in isolated ground recepticles (usually an...,12,[]
18843,i just installed a dx2-66 cpu in a clone mothe...,3,[]
18844,wouldn't this require a hyper-sphere. in 3-spa...,1,[]


In [30]:
df['tokenized_text'] = df.apply(lambda row: tokenizer.tokenize(row['Text']),axis=1)

In [31]:
df

,Text,Target,test,tokenized_text
0,i am sure some bashers of pens fans are pretty...,10,[],"[i, am, sure, some, bashers, of, pens, fans, a..."
1,my brother is in the market for a high-perform...,3,[],"[my, brother, is, in, the, market, for, a, hig..."
2,finally you said what you dream about. mediter...,17,[],"[finally, you, said, what, you, dream, about, ..."
3,think! it's the scsi card doing the dma transf...,3,[],"[think, it, s, the, scsi, card, doing, the, dm..."
4,1) i have an old drive which i cannot use with...,4,[],"[1, i, have, an, old, drive, which, i, cannot,..."
...,...,...,...,...
18841,dn> from: (david nye) dn> a neurology dn> cons...,13,[nyeda@cnsvax.uwec.edu ],"[dn, from, david, nye, dn, a, neurology, dn, c..."
18842,not in isolated ground recepticles (usually an...,12,[],"[not, in, isolated, ground, recepticles, usual..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,[],"[i, just, installed, a, dx2, 66, cpu, in, a, c..."
18844,wouldn't this require a hyper-sphere. in 3-spa...,1,[],"[wouldn, t, this, require, a, hyper, sphere, i..."


In [32]:
stop_words=set(stopwords.words("english"))

In [33]:
df['tokenized_text'] = df['tokenized_text'].apply(lambda x: [item for item in x if item not in stop_words])



In [34]:
df

,Text,Target,test,tokenized_text
0,i am sure some bashers of pens fans are pretty...,10,[],"[sure, bashers, pens, fans, pretty, confused, ..."
1,my brother is in the market for a high-perform...,3,[],"[brother, market, high, performance, video, ca..."
2,finally you said what you dream about. mediter...,17,[],"[finally, said, dream, mediterranean, new, are..."
3,think! it's the scsi card doing the dma transf...,3,[],"[think, scsi, card, dma, transfers, disks, scs..."
4,1) i have an old drive which i cannot use with...,4,[],"[1, old, drive, cannot, use, new, system, unde..."
...,...,...,...,...
18841,dn> from: (david nye) dn> a neurology dn> cons...,13,[nyeda@cnsvax.uwec.edu ],"[dn, david, nye, dn, neurology, dn, consultati..."
18842,not in isolated ground recepticles (usually an...,12,[],"[isolated, ground, recepticles, usually, unusu..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,[],"[installed, dx2, 66, cpu, clone, motherboard, ..."
18844,wouldn't this require a hyper-sphere. in 3-spa...,1,[],"[require, hyper, sphere, 3, space, 4, points, ..."


In [35]:
lemmatizer = WordNetLemmatizer()

In [36]:
df['lemmatized'] = df['tokenized_text'].apply(lambda x: [lemmatizer.lemmatize(y) for y in x])

In [37]:
df

,Text,Target,test,tokenized_text,lemmatized
0,i am sure some bashers of pens fans are pretty...,10,[],"[sure, bashers, pens, fans, pretty, confused, ...","[sure, bashers, pen, fan, pretty, confused, la..."
1,my brother is in the market for a high-perform...,3,[],"[brother, market, high, performance, video, ca...","[brother, market, high, performance, video, ca..."
2,finally you said what you dream about. mediter...,17,[],"[finally, said, dream, mediterranean, new, are...","[finally, said, dream, mediterranean, new, are..."
3,think! it's the scsi card doing the dma transf...,3,[],"[think, scsi, card, dma, transfers, disks, scs...","[think, scsi, card, dma, transfer, disk, scsi,..."
4,1) i have an old drive which i cannot use with...,4,[],"[1, old, drive, cannot, use, new, system, unde...","[1, old, drive, cannot, use, new, system, unde..."
...,...,...,...,...,...
18841,dn> from: (david nye) dn> a neurology dn> cons...,13,[nyeda@cnsvax.uwec.edu ],"[dn, david, nye, dn, neurology, dn, consultati...","[dn, david, nye, dn, neurology, dn, consultati..."
18842,not in isolated ground recepticles (usually an...,12,[],"[isolated, ground, recepticles, usually, unusu...","[isolated, ground, recepticles, usually, unusu..."
18843,i just installed a dx2-66 cpu in a clone mothe...,3,[],"[installed, dx2, 66, cpu, clone, motherboard, ...","[installed, dx2, 66, cpu, clone, motherboard, ..."
18844,wouldn't this require a hyper-sphere. in 3-spa...,1,[],"[require, hyper, sphere, 3, space, 4, points, ...","[require, hyper, sphere, 3, space, 4, point, s..."


In [38]:
df.lemmatized = df.lemmatized.apply(lambda x: " ".join(x) )

In [39]:
df

,Text,Target,test,tokenized_text,lemmatized
0,i am sure some bashers of pens fans are pretty...,10,[],"[sure, bashers, pens, fans, pretty, confused, ...",sure bashers pen fan pretty confused lack kind...
1,my brother is in the market for a high-perform...,3,[],"[brother, market, high, performance, video, ca...",brother market high performance video card sup...
2,finally you said what you dream about. mediter...,17,[],"[finally, said, dream, mediterranean, new, are...",finally said dream mediterranean new area grea...
3,think! it's the scsi card doing the dma transf...,3,[],"[think, scsi, card, dma, transfers, disks, scs...",think scsi card dma transfer disk scsi card dm...
4,1) i have an old drive which i cannot use with...,4,[],"[1, old, drive, cannot, use, new, system, unde...",1 old drive cannot use new system understandin...
...,...,...,...,...,...
18841,dn> from: (david nye) dn> a neurology dn> cons...,13,[nyeda@cnsvax.uwec.edu ],"[dn, david, nye, dn, neurology, dn, consultati...",dn david nye dn neurology dn consultation chea...
18842,not in isolated ground recepticles (usually an...,12,[],"[isolated, ground, recepticles, usually, unusu...",isolated ground recepticles usually unusual co...
18843,i just installed a dx2-66 cpu in a clone mothe...,3,[],"[installed, dx2, 66, cpu, clone, motherboard, ...",installed dx2 66 cpu clone motherboard tried m...
18844,wouldn't this require a hyper-sphere. in 3-spa...,1,[],"[require, hyper, sphere, 3, space, 4, points, ...",require hyper sphere 3 space 4 point specifies...


In [40]:
X,y = df.lemmatized,df.Target

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [42]:
str(X[0])

'sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end non pittsburghers relief bit praise pen man killing devil worse thought jagr showed much better regular season stats also lot fo fun watch playoff bowman let jagr lot fun next couple game since pen going beat pulp jersey anyway disappointed see islander lose final regular season game pen rule'

In [43]:
count_vectorizer = CountVectorizer(stop_words=None)

In [44]:
count_train = count_vectorizer.fit_transform(X_train)

In [45]:
count_train

<14134x84324 sparse matrix of type '<class 'numpy.int64'>'
	with 914894 stored elements in Compressed Sparse Row format>

In [46]:
df_BOW = pd.DataFrame(count_train.toarray(), columns = count_vectorizer.get_feature_names())
df_BOW

,00,000,0000,00000,00000000,0000000004,00000000b,00000001,00000001b,00000010,...,zz3,zz5co,zz9s,zzc6,zzd,zzf,zzvsi,zzz,zzzoh,zzzzzzt
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14130,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14131,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14132,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
df_BOW['00'].sum()

1778

In [48]:
df.test

0                                                       []
1                                                       []
2                                                       []
3                                                       []
4                                                       []
                               ...                        
18841                             [nyeda@cnsvax.uwec.edu ]
18842                                                   []
18843                                                   []
18844                                                   []
18845    [(crum@fcom.cc.utah.edu) , chriss@sam.amgen.co...
Name: test, Length: 18846, dtype: object

In [49]:
X[0]

'sure bashers pen fan pretty confused lack kind post recent pen massacre devil actually bit puzzled bit relieved however going put end non pittsburghers relief bit praise pen man killing devil worse thought jagr showed much better regular season stats also lot fo fun watch playoff bowman let jagr lot fun next couple game since pen going beat pulp jersey anyway disappointed see islander lose final regular season game pen rule'

In [50]:
df['Text'][18841].split()

['dn>',
 'from:',
 '(david',
 'nye)',
 'dn>',
 'a',
 'neurology',
 'dn>',
 'consultation',
 'is',
 'cheaper',
 'than',
 'a',
 'scan.',
 'and',
 'also',
 'better,',
 'because',
 'a',
 'neurologist',
 'can',
 'make',
 'a',
 'differential',
 'diagnosis',
 'between',
 'migraine,',
 'tension-type',
 'headache,',
 'cluster,',
 'benign',
 'intracranial',
 'hypertension,',
 'chronic',
 'paroxysmal',
 'hemicrania,',
 'and',
 'other',
 'headache',
 'syndromes',
 'that',
 'all',
 'appear',
 'normal',
 'on',
 'a',
 'scan.',
 'a',
 'neurologist',
 'can',
 'also',
 'recommend',
 'a',
 'course',
 'of',
 'treatment',
 'that',
 'is',
 'appropriate',
 'to',
 'the',
 'diagnosis.',
 'dn>',
 '>>also,',
 'since',
 'many',
 'people',
 'are',
 'convinced',
 'they',
 'have',
 'brain',
 'tumors',
 'or',
 'other',
 'dn>',
 '>>serious',
 'pathology,',
 'it',
 'may',
 'be',
 'cheaper',
 'to',
 'just',
 'get',
 'a',
 'ct',
 'scan',
 'then',
 'have',
 'dn>',
 '>>them',
 'come',
 'into',
 'the',
 'er',
 'every',
 'fe